In [ ]:
# default_exp build_parser


In [ ]:
#hide
from nbdev.showdoc import *


 # 03 build_parser Outlining a player's army, base and tech builds

 > : This module contains functions that allow parsing the players' build orders and some related performance indicators.

 ## Setup for examples and tests

 The following code contains this module's dependencies

In [ ]:
#exporti
from pathlib import Path
from pprint import pprint
from dataclasses import dataclass, astuple, field
from datetime import datetime
from typing import *
from fastcore import test as ft

import pandas as pd
import numpy as np
import csv
import json

import sc2reader

from sc2readertest import *

In [ ]:
#exporti
data_path = Path(Path.cwd()/'data') if Path('data').exists() else Path('../data')

with open(data_path/'unit_names.csv') as f:
    file_reader = csv.reader(f)
    unit_names = next(file_reader)
    
with open(data_path/'changes_names.csv') as f:
    file_reader = csv.reader(f)
    change_names = next(file_reader)
    
with open(data_path/'army_list.json') as f:
    race_armies = json.load(f)

with open(data_path/'buildings_list.json') as f:
    race_buildings = json.load(f)

with open(data_path/'upgrades.json') as f:
    race_upgrades = json.load(f)


In [ ]:
# The Replays are stored in the following dir.
rps_path = Path("./test_replays")

# single_replay is the base case I use to develop the functions in this module.
single_replay = sc2reader.load_replay(str(rps_path/"Jagannatha LE.SC2Replay"))

# The following replays have various compositions of races, armies, structures,
# tech updates and other game elements that allow me to test and debug the functions.abs
sing_zerg = sc2reader.load_replay(str(rps_path/"Oxide LE (14).SC2Replay"))
sing_protoss= sc2reader.load_replay(str(rps_path/"Oxide LE (13).SC2Replay"))
zustates = sc2reader.load_replay(str(rps_path/'zustates.SC2Replay'))
tustates = sc2reader.load_replay(str(rps_path/'tustates.SC2Replay'))
tfly = sc2reader.load_replay(str(rps_path/'terranfly.SC2Replay'))

# I store some basic variables out of the test case replay to make the sample code more
# readable.
match_events = [event for event in single_replay.events]
rpl_duration = single_replay.length.seconds
rpl_rec_duration = match_events[-1].second
rpl_fps = single_replay.game_fps


In [ ]:
def get_expansion_time(rpl: sc2reader.resources.Replay, pid: int) -> dict[str, float]:
    bases = {'Protoss':'nexus', 'Zerg':'hatchery', 'Terran':'commandcenter'}
    expansions = ['exp1','exp2','exp3','exp4']
    player_race = rpl.player[pid].play_race
    
    units_df = composition_df(rpl, pid, buildings=True)

    bases_df = units_df['Unit'].loc[(units_df.Unit == bases[player_race])]

    return units_df


In [ ]:
# units_df = sc2reader.load_replay(str(rps_path/'AllZergRepresented.SC2Replay'))
# print(get_expansion_time(zerg_test, 1))

 ### Player Tech Update
 Beyond buildings and training units, the third way players can spend their resources is by researching tech updates. 
However, contrary to units and buildings, player objects do not store a list of tech upgrades. Thus, I use the match's  `UpgradeCompleteEvent` to build this list. Bear in mind that, instead of recording a count for each upgrade, I can record the second at which the update takes place. I can do this because each type of update can only occur once every game. 
In the player profile, I can average the times a player researched a particular update, ignoring any matches when they didn't, to get an approximate measure of when they prefer to make any update. I can also count the number of times they research each upgrade to see what upgrades they favour. 


In [ ]:
#export
def list_player_upgrades(rpl: sc2reader.resources.Replay, pid: int) -> dict[str, float]:
    """Lists the times at wich the player completed their updates
    """
    player_race = rpl.player[pid].play_race

    upg_events = {upg_event.upgrade_type_name: calc_realtime_index(upg_event.second, rpl)
        for upg_event in rpl.events 
        if isinstance(upg_event, sc2reader.events.tracker.UpgradeCompleteEvent) 
        and upg_event.pid == pid
        and upg_event.upgrade_type_name in race_upgrades[player_race]}

    return {upgrade_name: upg_events.get(upgrade_name, 0) for upgrade_name in race_upgrades[player_race]}


In [ ]:
player1_upgrades = list_player_upgrades(sing_zerg, 1)
player1_upgrades, index=['Player1_upgrades']

{'BansheeCloak': 0,
 'BansheeSpeed': 0,
 'BattlecruiserEnableSpecializations': 0,
 'CycloneLockOnDamageUpgrade': 546.3041002277904,
 'DrillClaws': 427.7596810933941,
 'EnhancedShockwaves': 0,
 'HiSecAutoTracking': 0,
 'HighCapacityBarrels': 347.77790432801817,
 'LiberatorAGRangeUpgrade': 0,
 'MedivacIncreaseSpeedBoost': 0,
 'PersonalCloaking': 0,
 'PunisherGrenades': 0,
 'RavenCorvidReactor': 0,
 'ShieldWall': 0,
 'SmartServos': 526.3086560364465,
 'Stimpack': 0,
 'TerranBuildingArmor': 0,
 'TerranInfantryArmorsLevel1': 0,
 'TerranInfantryArmorsLevel2': 0,
 'TerranInfantryArmorsLevel3': 0,
 'TerranInfantryWeaponsLevel1': 0,
 'TerranInfantryWeaponsLevel2': 0,
 'TerranInfantryWeaponsLevel3': 0,
 'TerranShipWeaponsLevel1': 0,
 'TerranShipWeaponsLevel2': 0,
 'TerranShipWeaponsLevel3': 0,
 'TerranVehicleAndShipArmorsLevel1': 0,
 'TerranVehicleAndShipArmorsLevel2': 0,
 'TerranVehicleAndShipArmorsLevel3': 0,
 'TerranVehicleWeaponsLevel1': 574.1548974943053,
 'TerranVehicleWeaponsLevel2': 0,
 